This script will gather the data, prepare it for trainning, train an End-to-End ABSA by Exploiting BERT for End-to-End Aspect-based Sentiment Analysis (Xin Li et al, 2019) and predict the test dataset.

# Libraries

In [1]:
!git clone https://github.com/lixin4ever/BERT-E2E-ABSA/
!git clone https://github.com/GuilhermeMarcon/consentiment.git
!mv BERT-E2E-ABSA/* .

Cloning into 'BERT-E2E-ABSA'...
remote: Enumerating objects: 264, done.
remote: Counting objects: 100% (124/124), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 264 (delta 109), reused 104 (delta 104), pack-reused 140
Receiving objects: 100% (264/264), 1.61 MiB | 8.41 MiB/s, done.
Resolving deltas: 100% (141/141), done.
Cloning into 'consentiment'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 31 (delta 6), reused 6 (delta 1), pack-reused 0
Receiving objects: 100% (31/31), 869.78 KiB | 2.79 MiB/s, done.
Resolving deltas: 100% (6/6), done.


In [2]:
!pip install transformers
!pip3 install tensorboardX

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 3.5 MB/s eta 0:00:00


In [3]:
import pandas as pd
from ast import literal_eval
from html import unescape
from re import sub as strsub
import os

import numpy as np
import nltk
nltk.download('punkt')
from nltk.tokenize import TweetTokenizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Prepare Dataset

In [4]:
# don't change these:
COLUMN_SENTENCE = 'review'
COLUMN_ANNOTATION = 'annotation'

COLUMN_TAGS     = {'POS':'POS', 'NEU':'NEU', 'NEG':'NEG'}
COLUMN_TAGS_TO_ID = {'POS':0, 'NEU':1, 'NEG':2}


def read_datasets(filename, shuffle=True):
    if type(filename) is list:
        df = pd.concat([read_datasets(x) for x in filename], ignore_index=True)
        if shuffle: df = df.sample(frac=1).reset_index(drop=True)
        return df

    df = pd.read_excel(filename).rename(columns={'text':'review', 'snippet':'review'})
    df = df[['review', COLUMN_ANNOTATION]]
    df[COLUMN_ANNOTATION] = df[COLUMN_ANNOTATION].map(literal_eval)
    df = df[df[COLUMN_ANNOTATION].map(len) > 0]
    return df

def df_to_aux(df):
    data = []
    for index, row in df.iterrows():
        data.append([])
        data[-1].append(row[COLUMN_SENTENCE])
        data[-1].append([])
        data[-1].append([])
        data[-1].append([])

        for (aspect, sentiment) in row[COLUMN_ANNOTATION]:
            if sentiment in COLUMN_TAGS_TO_ID:
                data[-1][1+COLUMN_TAGS_TO_ID[sentiment]].append(aspect)
        for i in range(1, 4):
            data[-1][i] = '; '.join(data[-1][i])
    return pd.DataFrame(data, columns=['review', 'POS', 'NEU', 'NEG'])

def clean_text(text):
    # cleaning text, putting space between punctuations and dealing with "&#xx;" stuff
    # TODO it breaks emojis, fix?
    text = unescape(text)
    text = strsub('([.,!?()])', r' \1 ', text)
    text = strsub('\s{2,}', ' ', text)
    return text

def check_sequence(reqs, i, tokens):
    if len(tokens) == 0: return False
    return ' '.join(list(map(str.lower, reqs[i:i+len(tokens)]))) == ' '.join(list(map(str.lower, tokens)))

def find_reqs(reqs, tokens, req_name, tokenize):
    if str(tokens) == 'nan': tokens = None
    else: tokens = [tokenize(clean_text(feat)) for feat in str(tokens).split(';')]
    if tokens is None: return False

    for i in range(0, len(reqs[0])):
        for tk in tokens:
            if check_sequence(reqs[0], i, tk):
                reqs[1][i] = 'B-'+req_name
                for j in range(1, len(tk)):
                    reqs[1][i+j] = 'I-'+req_name
    return True

def format_reqs(text, reqs, tags):
    if not isinstance(text, str): return None
    text = clean_text(text)

    # word_tokenize separates can't and similar as "ca" and "n't"
    # so i'm using TweetTokenizer since its casual user reviews
    tokenize = TweetTokenizer().tokenize
    # tokenize = nltk.word_tokenize
    text_tk = tokenize(text)

    ret_reqs = []
    for text_tk_i in text_tk:
        sub_text_tk = text_tk_i.split(' ') # dealing with edge case of one token being: '. . .'

        for sub_i in sub_text_tk:
            ret_reqs.append([sub_i, 'O'])
    ret_reqs = np.asarray(ret_reqs).T.tolist()

    [find_reqs(ret_reqs, reqs[i], tags[i], tokenize) for i in range(len(reqs))]

    return np.asarray(ret_reqs).T.tolist()

# Usage: Tuple array ["Column name", "Entity tag"]
# e.g. [['Feature (Positive)', 'POS], ['Feature (Neutral)', 'NTL], ['Feature (Negative)', 'NEG]]
def format_reqs_dataset(df, name_tuples):
    name_tuples = np.asarray(name_tuples).T
    return list(map(lambda sentences, entities: format_reqs(sentences, entities, name_tuples[1]),
                    df[COLUMN_SENTENCE].values, df[name_tuples[0]].values))

def write_e2eabsa(filename, df):
    with open(filename, 'w') as f:
        for idx, row in df.iterrows():
            f.write(row[COLUMN_SENTENCE]+"####")
            for idx2, ner in enumerate(row['ner']):
                if(idx2 > 0): f.write(' ')
                tag = 'O'
                if ner[1] == 'B-POS' or ner[1] == 'I-POS': tag = 'T-POS'
                elif ner[1] == 'B-NTL' or ner[1] == 'I-NTL': tag = 'T-NEU'
                elif ner[1] == 'B-NEG' or ner[1] == 'I-NEG': tag = 'T-NEG'
                f.write(ner[0]+'='+tag)
            f.write('\n')

def df_to_e2eabsa(df, filename):
    aux = df_to_aux(df)
    # display(aux)
    aux['ner'] = format_reqs_dataset(aux, list(COLUMN_TAGS.items()))
    write_e2eabsa(filename, aux)

In [6]:
# possible datasets: laptop14, rest14 and hotel22
DATASET = 'hotel22'

# don't change these
COLUMN_ANNOTATION = 'annotation'
ROOT = '/content/consentiment/data/'+DATASET+'/'

import os

try: os.mkdir('/content/data/hotel22/')
except: pass

for filename in ['train', 'dev', 'test']:
    E2E_ABSA_FILENAME = '/content/data/'+DATASET+'/'+filename+'.txt'
    df = read_datasets(ROOT+filename+'.xlsx')
    df_to_e2eabsa(df, E2E_ABSA_FILENAME)
df

,review,annotation
0,Em abril de 2012 nos hospedamos no hotel refer...,"[(lojas, POS), (hotel, POS), (quartos, POS), (..."
1,"EU fiquei uma semana neste hotel, é maravilhos...","[(rua, NEU), (café da manhã, POS), (localizaçã..."
2,"Localização excelente, atendimento e serviços ...","[(cama, NEU), (serviços, POS), (Café da manhã,..."
3,Muito satisfeita com minha escolha de hospedag...,"[(localização, POS), (piscinas, POS)]"
4,Estive hospedado no Mandarin Oriental NYC este...,"[(piscina, POS), (serviço, POS), (cama, POS), ..."
...,...,...
79,"The Benjamim hotel possui bom atendimento, qua...","[(atendimento, POS), (quarto, POS), (hotel, PO..."
80,"Me hospedei nesse hotel em março de 2012, pega...","[(frigobar, NEG), (quarto, POS), (hotel, NEU)]"
81,É um hotel com uma excelente localização. Em f...,"[(localização, POS), (almoço, POS)]"
82,"Primeiro ponto: A localização é muito boa, fac...","[(quartos, NEU), (localização, POS), (corredor..."


# Train E2E-ABSA model

In [7]:
# possible values: bert-base-uncased, bert-base-multilingual-cased
# it is possible to use other bert models, I refer that to https://github.com/lixin4ever/BERT-E2E-ABSA
BERT_MODEL = 'bert-base-multilingual-cased'
# possible values: linear, san, tfm, crf, gru
E2EABSA_TYPE = 'san'

# don't change these
script = """import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0, 3, 1, 2"

#seed_numbers = [42, 593, 1774, 65336, 189990]
seed_numbers = [42]
model_type = 'bert'
absa_type = '"""+E2EABSA_TYPE+"""'
tfm_mode = 'finetune'
fix_tfm = 0
data_folder = '"""+DATASET+"""'
task_name = data_folder
warmup_steps = 0
overfit = 0
if task_name == 'laptop14':
    train_batch_size = 32
else:
    task_name = 'rest14'
    train_batch_size = 16

for run_id, seed in enumerate(seed_numbers):
    command = "python main.py --model_type %s --absa_type %s --tfm_mode %s --fix_tfm %s " \
              "--model_name_or_path """+BERT_MODEL+""" --data_dir ./data/%s --task_name %s " \
              "--per_gpu_train_batch_size %s --per_gpu_eval_batch_size 8 --learning_rate 2e-5 " \
              "--max_steps 1500 --warmup_steps %s --do_train --do_eval " \
              "--seed %s --tagging_schema BIEOS --overfit %s " \
              "--overwrite_output_dir --eval_all_checkpoints --MASTER_ADDR localhost --MASTER_PORT 28512" % (
        model_type, absa_type, tfm_mode, fix_tfm, data_folder, task_name, train_batch_size, warmup_steps, seed, overfit)
    output_dir = '%s-%s-%s-%s' % (model_type, absa_type, task_name, tfm_mode)
    if fix_tfm:
        output_dir = '%s-fix' % output_dir
    if overfit:
        output_dir = '%s-overfit' % output_dir
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)

    log_file = '%s/log.txt' % output_dir
    if run_id == 0 and os.path.exists(log_file):
        os.remove(log_file)
    with open(log_file, 'a') as fp:
        fp.write("In run %s/5 (seed %s): " % (run_id, seed))
    os.system(command)
    if overfit:
        # only conduct one run
        break
"""

f = open("fast_run.py", "w")
f.write(script)
f.close()

In [8]:
!python fast_run.py

10/26/2023 10:53:15 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
Some weights of BertABSATagger were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['tagger.self_attn.in_proj_bias', 'tagger.norm.bias', 'tagger.self_attn.out_proj.weight', 'tagger.norm.weight', 'tagger.self_attn.out_proj.bias', 'tagger.self_attn.in_proj_weight', 'classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
train class count: [2015.  502.    0.]
10/26/2023 10:53:31 - INFO - glue_utils -   *** Example ***
10/26/2023 10:53:31 - INFO - glue_utils -   guid: train-676
10/26/2023 10:53:31 - INFO - glue_utils -   tokens: [CLS] Em primeiro lugar dizer que é um hotel 4 estrelas com serviço de 5 estrelas ! Ó ##tima localiza ##ção , 2 minutos a pé do metro St - A ##gust ##in , próximo das ruas mais bal

In [9]:
# Saving best checkpoint, change these:
ROOT = '/content/models/'
FOLDER = 'hotel22'
# the best checkpoint should be explicit in the output of the previous cell
CHECKPOINT = 1300


!mkdir {ROOT}
!mkdir {ROOT}{FOLDER}
!mkdir {ROOT}{FOLDER}/{BERT_MODEL}-e2eabsa-{E2EABSA_TYPE}
!cp bert-{E2EABSA_TYPE}-rest14-finetune/* {ROOT}{FOLDER}/{BERT_MODEL}-e2eabsa-{E2EABSA_TYPE}
if CHECKPOINT != 1500:
    !cp bert-{E2EABSA_TYPE}-rest14-finetune/checkpoint-{str(CHECKPOINT)}/* {ROOT}{FOLDER}/{BERT_MODEL}-e2eabsa-{E2EABSA_TYPE}

cp: -r not specified; omitting directory 'bert-san-rest14-finetune/checkpoint-100'
cp: -r not specified; omitting directory 'bert-san-rest14-finetune/checkpoint-1000'
cp: -r not specified; omitting directory 'bert-san-rest14-finetune/checkpoint-1100'
cp: -r not specified; omitting directory 'bert-san-rest14-finetune/checkpoint-1200'
cp: -r not specified; omitting directory 'bert-san-rest14-finetune/checkpoint-1300'
cp: -r not specified; omitting directory 'bert-san-rest14-finetune/checkpoint-1400'
cp: -r not specified; omitting directory 'bert-san-rest14-finetune/checkpoint-1500'
cp: -r not specified; omitting directory 'bert-san-rest14-finetune/checkpoint-200'
cp: -r not specified; omitting directory 'bert-san-rest14-finetune/checkpoint-300'
cp: -r not specified; omitting directory 'bert-san-rest14-finetune/checkpoint-400'
cp: -r not specified; omitting directory 'bert-san-rest14-finetune/checkpoint-500'
cp: -r not specified; omitting directory 'bert-san-rest14-finetune/checkpoint-600

# Predict

In [10]:
from transformers import BertTokenizer
from absa_layer import BertABSATagger
from work import load_and_cache_examples
from work import predict as predict_base
import torch

# to test on a new dataset, you can change the "test.txt" from "/content/data/DATASET/"
# make sure to leave in the same format as https://github.com/lixin4ever/BERT-E2E-ABSA

# DATASET = 'hotel22'
ROOT_FOLDER = ROOT+FOLDER+'/'
TASK_NAME = 'laptop14' if DATASET == 'laptop14' else 'rest14'

class FakeArguments():
    def __init__(self, absa_home, ckpt, data_dir, task_name, model_type='bert',
                 model_name_or_path='bert-base-uncased', cache_dir='./cache',
                 max_seq_length=128, tagging_schema='BIEOS'):
        self.absa_home = absa_home
        self.ckpt = absa_home+('/' if not absa_home.endswith('/') else '')+ckpt
        self.data_dir = './data/'+data_dir
        self.task_name = task_name
        self.model_type = model_type
        self.model_name_or_path = model_name_or_path
        self.cache_dir = cache_dir
        self.max_seq_length = max_seq_length
        self.tagging_schema = tagging_schema

# args = FakeArguments(absa_home="./bert-san-rest14-finetune", ckpt='checkpoint-1400',
#                      model_name_or_path='bert-base-multilingual-cased',
#                      data_dir='hotel22', task_name='rest14', tagging_schema='BIEOS')
args = FakeArguments(absa_home=ROOT_FOLDER+BERT_MODEL+"-e2eabsa-"+E2EABSA_TYPE,
                     ckpt='',
                     model_name_or_path=BERT_MODEL,
                     data_dir=DATASET, task_name=TASK_NAME, tagging_schema='BIEOS')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
args.device = device
if torch.cuda.is_available():
    args.n_gpu = torch.cuda.device_count()

model = BertABSATagger.from_pretrained(args.ckpt)
tokenizer = BertTokenizer.from_pretrained(args.absa_home)
model.to(args.device)
model.eval()

BertABSATagger(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [11]:
from tqdm import tqdm
import numpy as np
from torch.utils.data import DataLoader, SequentialSampler
from seq_utils import ot2bieos_ts, bio2ot_ts, tag2ts, ot2bio_ts

def bieos2ot_ts(ts_tag_sequence): # same as bio2ot_ts
    """
    perform bieos-->ot for ts tag sequence
    :param ts_tag_sequence:
    :return:
    """
    new_ts_sequence = []
    n_tags = len(ts_tag_sequence)
    for i in range(n_tags):
        ts_tag = ts_tag_sequence[i]
        if ts_tag == 'O' or ts_tag == 'EQ':
            new_ts_sequence.append('O')
        else:
            pos, sentiment = ts_tag.split('-')
            new_ts_sequence.append('T-%s' % sentiment)
    return new_ts_sequence

# code from work.py, changed to return BIO tag predictions for nervaluate
def predict(args, model, tokenizer):
    dataset, evaluate_label_ids, total_words = load_and_cache_examples(args, args.task_name, tokenizer)

    sampler = SequentialSampler(dataset)
    # process the incoming data one by one
    dataloader = DataLoader(dataset, sampler=sampler, batch_size=1)
    # print("***** Running prediction *****")

    if args.tagging_schema == 'BIEOS':
        absa_label_vocab = {'O': 0, 'EQ': 1, 'B-POS': 2, 'I-POS': 3, 'E-POS': 4, 'S-POS': 5,
                        'B-NEG': 6, 'I-NEG': 7, 'E-NEG': 8, 'S-NEG': 9,
                        'B-NEU': 10, 'I-NEU': 11, 'E-NEU': 12, 'S-NEU': 13}
    elif args.tagging_schema == 'BIO':
        absa_label_vocab = {'O': 0, 'EQ': 1, 'B-POS': 2, 'I-POS': 3,
        'B-NEG': 4, 'I-NEG': 5, 'B-NEU': 6, 'I-NEU': 7}
    elif args.tagging_schema == 'OT':
        absa_label_vocab = {'O': 0, 'EQ': 1, 'T-POS': 2, 'T-NEG': 3, 'T-NEU': 4}
    else:
        raise Exception("Invalid tagging schema %s..." % args.tagging_schema)
    absa_id2tag = {}
    for k in absa_label_vocab:
        v = absa_label_vocab[k]
        absa_id2tag[v] = k

    total_preds, gold_labels = None, None
    idx = 0
    predictions = []

    for batch in tqdm(dataloader, desc="Evaluating"):
        batch = tuple(t.to(args.device) for t in batch)
        with torch.no_grad():
            inputs = {'input_ids': batch[0],
                      'attention_mask': batch[1],
                      'token_type_ids': batch[2] if args.model_type in ['bert', 'xlnet'] else None,
                      # XLM don't use segment_ids
                      'labels': batch[3]}
            outputs = model(**inputs)
            logits = outputs[1]
            if model.tagger_config.absa_type != 'crf':
                preds = np.argmax(logits.detach().cpu().numpy(), axis=-1)
            else:
                mask = batch[1]
                preds = model.tagger.viterbi_tags(logits=logits, mask=mask)

            label_indices = evaluate_label_ids[idx]
            words = total_words[idx]
            pred_labels = preds[0][label_indices]
            assert len(words) == len(pred_labels)

            pred_tags = [absa_id2tag[label] for label in pred_labels]
            if args.tagging_schema == 'OT':
                pred_tags = ot2bieos_ts(pred_tags)
            elif args.tagging_schema == 'BIO':
                pred_tags = ot2bieos_ts(bio2ot_ts(pred_tags))
            else:
                # current tagging schema is BIEOS, do nothing
                pass

            p_ts_sequence = tag2ts(ts_tag_sequence=pred_tags)

            output_ts = []
            predictions.append([])
            for t in p_ts_sequence:
                beg, end, sentiment = t
                aspect = words[beg:end+1]
                predictions[-1].append((' '.join(aspect), sentiment))
                output_ts.append('%s: %s' % (aspect, sentiment))
            if inputs['labels'] is not None:
                # for the unseen data, there is no ``labels''
                if gold_labels is None:
                    gold_labels = inputs['labels'].detach().cpu().numpy()
                else:
                    gold_labels = np.append(gold_labels, inputs['labels'].detach().cpu().numpy(), axis=0)
        idx += 1
    return predictions

predictions = predict(args, model, tokenizer)
predictions

cached_features_file: ./data/hotel22/cached_test_bert-base-multilingual-cased_128_rest14
test class count: [266.  50.   0.]


Evaluating: 100%|██████████| 84/84 [00:02<00:00, 31.03it/s]


[[('hotel', 'POS'), ('limpeza', 'POS'), ('custo-benefício', 'POS')],
 [('hotel', 'POS'),
  ('localização', 'POS'),
  ('hotel', 'POS'),
  ('antendimento', 'POS'),
  ('café da manhã', 'POS'),
  ('rua', 'POS')],
 [('Localização', 'POS'),
  ('atendimento', 'POS'),
  ('serviços', 'POS'),
  ('hotel', 'POS'),
  ('quarto', 'POS'),
  ('cama', 'POS'),
  ('cama', 'POS'),
  ('Quarto', 'POS'),
  ('hotel', 'POS')],
 [('Hotel', 'POS'),
  ('hotel', 'POS'),
  ('Hotel', 'POS'),
  ('localização', 'POS'),
  ('quarto', 'POS'),
  ('hotel', 'POS'),
  ('preço', 'POS'),
  ('piscinas', 'POS'),
  ('hotel', 'POS')],
 [('quarto', 'POS'),
  ('cama', 'POS'),
  ('banheira', 'POS'),
  ('serviço', 'POS'),
  ('piscina', 'POS')],
 [('Hotel', 'POS'), ('cama', 'POS'), ('internet', 'POS')],
 [('quarto', 'POS'),
  ('preço', 'POS'),
  ('localização', 'POS'),
  ('chuveiro', 'POS'),
  ('atendimento', 'POS')],
 [('localização', 'POS'),
  ('elevador', 'NEG'),
  ('quartos', 'NEG'),
  ('estacionamento', 'NEG')],
 [('quarto', 'POS')

# Add to results table

In [12]:
import pandas as pd
from ast import literal_eval

TEST_FILENAME = '/content/consentiment/data/'+DATASET+'/test.xlsx'

print('Reading', TEST_FILENAME)
df = pd.read_excel(TEST_FILENAME)
ENTS_COLUMN = 1

for c in df.columns[ENTS_COLUMN:]:
    print(c)
    df[c] = df[c].apply(literal_eval)
df

Reading /content/consentiment/data/hotel22/test.xlsx
annotation
hotel22-bert-base-multilingual-cased-e2eabsa-linear
hotel22-bert-base-multilingual-cased-e2eabsa-san
hotel22-bert-base-multilingual-cased-e2eabsa-tfm
hotel22-bert-base-multilingual-cased-e2eabsa-crf
hotel22-bert-base-multilingual-cased-e2eabsa-gru
committee-voting
rest14-bert-base-multilingual-cased-e2eabsa-linear
rest14-bert-base-multilingual-cased-e2eabsa-san
rest14-bert-base-multilingual-cased-e2eabsa-tfm
rest14-bert-base-multilingual-cased-e2eabsa-crf
rest14-bert-base-multilingual-cased-e2eabsa-gru
graph-bert-base-multilingual-cased-e2eabsa-linear
graph-bert-base-multilingual-cased-e2eabsa-san
graph-bert-base-multilingual-cased-e2eabsa-tfm
graph-bert-base-multilingual-cased-e2eabsa-crf
graph-bert-base-multilingual-cased-e2eabsa-gru
committee-graph
multidomain-bert-base-multilingual-cased-e2eabsa-linear
multidomain-bert-base-multilingual-cased-e2eabsa-san
multidomain-bert-base-multilingual-cased-e2eabsa-tfm
multidomain-

,review,annotation,hotel22-bert-base-multilingual-cased-e2eabsa-linear,hotel22-bert-base-multilingual-cased-e2eabsa-san,hotel22-bert-base-multilingual-cased-e2eabsa-tfm,hotel22-bert-base-multilingual-cased-e2eabsa-crf,hotel22-bert-base-multilingual-cased-e2eabsa-gru,committee-voting,rest14-bert-base-multilingual-cased-e2eabsa-linear,rest14-bert-base-multilingual-cased-e2eabsa-san,...,multidomain-bert-base-multilingual-cased-e2eabsa-san,multidomain-bert-base-multilingual-cased-e2eabsa-tfm,multidomain-bert-base-multilingual-cased-e2eabsa-crf,multidomain-bert-base-multilingual-cased-e2eabsa-gru,gpt3.5,gpt3.5-bert-base-multilingual-cased-e2eabsa-linear,gpt3.5-bert-base-multilingual-cased-e2eabsa-san,gpt3.5-bert-base-multilingual-cased-e2eabsa-tfm,gpt3.5-bert-base-multilingual-cased-e2eabsa-crf,gpt3.5-bert-base-multilingual-cased-e2eabsa-gru
0,Em abril de 2012 nos hospedamos no hotel refer...,"[(lojas, POS), (hotel, POS), (quartos, POS), (...","[(hotel, POS), (localizacao, POS), (farmacia, ...","[(hotel, POS), (localizacao, POS), (farmacia, ...","[(hotel, POS), (localizacao, POS), (farmacia, ...","[(localizacao, POS), (lojas, POS), (Funcionari...","[(hotel, POS), (localizacao, POS), (farmacia, ...","[(hotel, POS), (localizacao, POS), (farmacia, ...","[(localizacao, POS), (farmacia, POS), (restaur...","[(localizacao, POS), (farmacia, POS), (restaur...",...,"[(localizacao, POS), (farmacia, POS), (lojas, ...","[(localizacao, POS), (farmacia, POS), (lojas, ...","[(localizacao, POS), (farmacia, POS), (restaur...","[(localizacao, POS), (farmacia, POS), (lojas, ...","[(localizacao, POS), (metro, POS), (farmacia, ...","[(localizacao, POS), (farmacia, POS), (Funcion...","[(hotel, POS), (localizacao, POS), (farmacia, ...","[(hotel, POS), (localizacao, POS), (farmacia, ...","[(localizacao, POS), (farmacia, POS), (restaur...","[(localizacao, POS), (farmacia, POS), (restaur..."
1,"EU fiquei uma semana neste hotel, é maravilhos...","[(rua, NEU), (café da manhã, POS), (localizaçã...","[(hotel, POS), (localização, POS), (antendimen...","[(hotel, POS), (localização, POS), (hotel, POS...","[(hotel, POS), (localização, POS), (hotel, POS...","[(hotel, POS), (localização, POS), (hotel, POS...","[(hotel, POS), (localização, POS), (hotel, POS...","[(hotel, POS), (localização, POS), (antendimen...","[(localização, POS), (louvre, POS), (metro, PO...","[(localização, POS), (antendimento, POS), (caf...",...,"[(localização, POS), (louvre, POS), (antendime...","[(localização, POS), (louvre, POS), (moulan, P...","[(localização, POS), (louvre, POS), (moulan, P...","[(localização, POS), (antendimento, POS), (caf...","[(hotel, POS), (localização, POS), (atendiment...","[(hotel, POS), (localização, POS), (louvre, PO...","[(hotel, POS), (localização, POS), (louvre, PO...","[(hotel, POS), (localização, POS), (caminhar, ...","[(hotel, POS), (localização, POS), (moulan rou...","[(hotel, POS), (localização, POS), (louvre, PO..."
2,"Localização excelente, atendimento e serviços ...","[(cama, NEU), (serviços, POS), (Café da manhã,...","[(Localização, POS), (atendimento, POS), (serv...","[(Localização, POS), (atendimento, POS), (serv...","[(Localização, POS), (atendimento, POS), (serv...","[(Localização, POS), (atendimento, POS), (serv...","[(Localização, POS), (atendimento, POS), (serv...","[(Localização, POS), (atendimento, POS), (serv...","[(Localização, POS), (atendimento, POS), (serv...","[(Localização, POS), (atendimento, POS), (serv...",...,"[(Localização, POS), (atendimento, POS), (serv...","[(Localização, POS), (atendimento, POS), (serv...","[(Localização, POS), (atendimento, POS), (serv...","[(Localização, POS), (quarto, POS), (banheiro,...","[(Localização, POS), (atendimento, POS), (serv...","[(Localização, POS), (atendimento, POS), (serv...","[(Localização, POS), (atendimento, POS), (serv...","[(Localização, POS), (atendimento, POS), (serv...","[(Localização, POS), (atendimento, POS), (serv...","[(Localização, POS), (atendimento, POS), (serv..."
3,Muito satisfe

In [13]:
DF_PREFIX = DATASET
df[DF_PREFIX+'-'+BERT_MODEL+'-e2eabsa-'+E2EABSA_TYPE] = predictions
df

,review,annotation,hotel22-bert-base-multilingual-cased-e2eabsa-linear,hotel22-bert-base-multilingual-cased-e2eabsa-san,hotel22-bert-base-multilingual-cased-e2eabsa-tfm,hotel22-bert-base-multilingual-cased-e2eabsa-crf,hotel22-bert-base-multilingual-cased-e2eabsa-gru,committee-voting,rest14-bert-base-multilingual-cased-e2eabsa-linear,rest14-bert-base-multilingual-cased-e2eabsa-san,...,multidomain-bert-base-multilingual-cased-e2eabsa-san,multidomain-bert-base-multilingual-cased-e2eabsa-tfm,multidomain-bert-base-multilingual-cased-e2eabsa-crf,multidomain-bert-base-multilingual-cased-e2eabsa-gru,gpt3.5,gpt3.5-bert-base-multilingual-cased-e2eabsa-linear,gpt3.5-bert-base-multilingual-cased-e2eabsa-san,gpt3.5-bert-base-multilingual-cased-e2eabsa-tfm,gpt3.5-bert-base-multilingual-cased-e2eabsa-crf,gpt3.5-bert-base-multilingual-cased-e2eabsa-gru
0,Em abril de 2012 nos hospedamos no hotel refer...,"[(lojas, POS), (hotel, POS), (quartos, POS), (...","[(hotel, POS), (localizacao, POS), (farmacia, ...","[(hotel, POS), (limpeza, POS), (custo-benefíci...","[(hotel, POS), (localizacao, POS), (farmacia, ...","[(localizacao, POS), (lojas, POS), (Funcionari...","[(hotel, POS), (localizacao, POS), (farmacia, ...","[(hotel, POS), (localizacao, POS), (farmacia, ...","[(localizacao, POS), (farmacia, POS), (restaur...","[(localizacao, POS), (farmacia, POS), (restaur...",...,"[(localizacao, POS), (farmacia, POS), (lojas, ...","[(localizacao, POS), (farmacia, POS), (lojas, ...","[(localizacao, POS), (farmacia, POS), (restaur...","[(localizacao, POS), (farmacia, POS), (lojas, ...","[(localizacao, POS), (metro, POS), (farmacia, ...","[(localizacao, POS), (farmacia, POS), (Funcion...","[(hotel, POS), (localizacao, POS), (farmacia, ...","[(hotel, POS), (localizacao, POS), (farmacia, ...","[(localizacao, POS), (farmacia, POS), (restaur...","[(localizacao, POS), (farmacia, POS), (restaur..."
1,"EU fiquei uma semana neste hotel, é maravilhos...","[(rua, NEU), (café da manhã, POS), (localizaçã...","[(hotel, POS), (localização, POS), (antendimen...","[(hotel, POS), (localização, POS), (hotel, POS...","[(hotel, POS), (localização, POS), (hotel, POS...","[(hotel, POS), (localização, POS), (hotel, POS...","[(hotel, POS), (localização, POS), (hotel, POS...","[(hotel, POS), (localização, POS), (antendimen...","[(localização, POS), (louvre, POS), (metro, PO...","[(localização, POS), (antendimento, POS), (caf...",...,"[(localização, POS), (louvre, POS), (antendime...","[(localização, POS), (louvre, POS), (moulan, P...","[(localização, POS), (louvre, POS), (moulan, P...","[(localização, POS), (antendimento, POS), (caf...","[(hotel, POS), (localização, POS), (atendiment...","[(hotel, POS), (localização, POS), (louvre, PO...","[(hotel, POS), (localização, POS), (louvre, PO...","[(hotel, POS), (localização, POS), (caminhar, ...","[(hotel, POS), (localização, POS), (moulan rou...","[(hotel, POS), (localização, POS), (louvre, PO..."
2,"Localização excelente, atendimento e serviços ...","[(cama, NEU), (serviços, POS), (Café da manhã,...","[(Localização, POS), (atendimento, POS), (serv...","[(Localização, POS), (atendimento, POS), (serv...","[(Localização, POS), (atendimento, POS), (serv...","[(Localização, POS), (atendimento, POS), (serv...","[(Localização, POS), (atendimento, POS), (serv...","[(Localização, POS), (atendimento, POS), (serv...","[(Localização, POS), (atendimento, POS), (serv...","[(Localização, POS), (atendimento, POS), (serv...",...,"[(Localização, POS), (atendimento, POS), (serv...","[(Localização, POS), (atendimento, POS), (serv...","[(Localização, POS), (atendimento, POS), (serv...","[(Localização, POS), (quarto, POS), (banheiro,...","[(Localização, POS), (atendimento, POS), (serv...","[(Localização, POS), (atendimento, POS), (serv...","[(Localização, POS), (atendimento, POS), (serv...","[(Localização, POS), (atendimento, POS), (serv...","[(Localização, POS), (atendimento, POS), (serv...","[(Localização, POS), (atendimento, POS), (serv..."
3,Muito satisfe

In [14]:
df.to_excel(TEST_FILENAME, index=False)